In [2]:
def load_data_and_labels(data_folder, num_classes, img_size=(28, 28), color_mode='grayscale'):
    X = []
    y_onehot = []
    
    for idx, class_folder in enumerate(sorted(os.listdir(data_folder))):
        class_path = os.path.join(data_folder, class_folder)
        
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = load_img(img_path, target_size=img_size, color_mode=color_mode)
            X.append(img_to_array(img))
            
            onehot = np.zeros(num_classes)
            onehot[idx] = 1
            y_onehot.append(onehot)
    
    return np.array(X) / 255.0, np.array(y_onehot)

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 절반씩 줄여보기도 하기

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolution 1
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())

    # Maxpooling 1
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 2
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 2
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 3
    model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 3
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten
    model.add(layers.Flatten())

    # FC 1
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.5))

    # FC 2
    model.add(layers.Dense(2048, activation='relu')) 
    model.add(layers.Dropout(0.5))

    # FC 3 (출력층)
    model.add(layers.Dense(num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model

In [4]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time

# 데이터 경로
base_path = r'C:\Users\pc\Desktop\CNN\36_feature_multiclass_CNN'
train_data_folder = os.path.join(base_path, 'Train_Image')
val_data_folder = os.path.join(base_path, 'Validation_Image')
test_data_folder = os.path.join(base_path, 'Test_Image')

num_classes = 11

# 데이터 로드
X_train, y_train_onehot = load_data_and_labels(train_data_folder, num_classes)
X_val, y_val_onehot = load_data_and_labels(val_data_folder, num_classes)
X_test, y_test_onehot = load_data_and_labels(test_data_folder, num_classes)

# 모델 생성
input_shape = X_train[0].shape
model = create_model(input_shape, num_classes)

# 모델 학습을 위한 배치 사이즈를 설정
batch_size = 128  # 로 고정하는게 좋을 것 같음

# 모델 학습
epochs = 50
history = model.fit(X_train, y_train_onehot, batch_size=batch_size, validation_data=(X_val, y_val_onehot), epochs=epochs)

# 시간 측정 시작
start_time = time.time()

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(X_test, y_test_onehot)

# 시간 측정 종료
end_time = time.time()

# 걸린 시간 계산
elapsed_time = end_time - start_time

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)
print(f"Test Duration: {elapsed_time:.4f} seconds")  # 소수점 4자리까지 출력

Epoch 1/50
2500/2500 [==============================] - 28s 9ms/step - loss: 1.7935 - accuracy: 0.5005 - auc: 0.7848 - val_loss: 1.7569 - val_accuracy: 0.5028 - val_auc: 0.7989
Epoch 2/50
2500/2500 [==============================] - 22s 9ms/step - loss: 1.7296 - accuracy: 0.5069 - auc: 0.8072 - val_loss: 1.6953 - val_accuracy: 0.5129 - val_auc: 0.8182
Epoch 3/50
2500/2500 [==============================] - 22s 9ms/step - loss: 1.6863 - accuracy: 0.5128 - auc: 0.8208 - val_loss: 1.6666 - val_accuracy: 0.5149 - val_auc: 0.8280
Epoch 4/50
2500/2500 [==============================] - 22s 9ms/step - loss: 1.6659 - accuracy: 0.5146 - auc: 0.8272 - val_loss: 1.6545 - val_accuracy: 0.5167 - val_auc: 0.8328
Epoch 5/50
2500/2500 [==============================] - 22s 9ms/step - loss: 1.6529 - accuracy: 0.5163 - auc: 0.8313 - val_loss: 1.6429 - val_accuracy: 0.5170 - val_auc: 0.8346
Epoch 6/50
2500/2500 [==============================] - 22s 9ms/step - loss: 1.6422 - accuracy: 0.5170 - auc: 0.834

In [5]:
from sklearn.metrics import confusion_matrix
import numpy as np

# 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 다중 클래스 분류 문제에서는 가장 높은 확률을 가진 클래스를 선택합니다.

# 실제 레이블을 가져옵니다.
y_true = np.argmax(y_test_onehot, axis=1)

# Confusion Matrix를 생성합니다.
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# 정확도를 계산합니다.
accuracy = np.trace(conf_matrix) / float(np.sum(conf_matrix))

print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)

1250/1250 [==============================] - 2s 1ms/step
Confusion Matrix:
[[19843    16     1    31     0    13    23    14    17    41     1]
 [ 2008   128     4    26     2    11    35     2     1    30     0]
 [ 1346    19    10    19     0    11    26     5     0    27     0]
 [ 2195    17     8   289     1     8    45     3     1    25     1]
 [ 1366    17    10    22     0     5    28     2     1    22     0]
 [  122     3     0     0     0   120     1     2     2    35     0]
 [ 2239    19     7    41     1     9   229     3     0    31     0]
 [ 2166     3     0     6     0     3     9    83    47    27     6]
 [ 1865     2     0     1     0     1     5    64    74    35     9]
 [ 2622     8     0    14     1     7    20    16    17   430     4]
 [ 1649     0     0     4     0     3     3    80    36    28    12]]
Accuracy: 0.53045
